In [11]:
%%cython
import numpy as np
cimport numpy as np
from scipy.optimize import minimize
from libc.math cimport isnan
from libc.math cimport fabs
cimport cython.dataclasses.dataclass
cimport cython.dataclasses.field
from numpy cimport ndarray

@cython.boundscheck(False)
@cython.wraparound(False)
cdef np.ndarray[np.float64_t, ndim=1] cython_dot_with_nans(np.ndarray[np.float64_t, ndim=1] Q, np.ndarray[np.float64_t, ndim=1] x):
    cdef np.ndarray[np.float64_t, ndim=1] Z = Q.copy()
    cdef np.ndarray[np.float64_t, ndim=1] nans_array = np.nansum(Z*Q+x, axis=1)
    cdef Py_ssize_t i
    for i in range(Z.shape[0]):
        Z[i, len(Z)-1] = (Z[i, len(Z)-1].T * (1/(nans_array[i]+1))).T

    cdef np.ndarray[np.float64_t, ndim=1] weighted_sum = np.nansum(Z*x, axis=1)
    cdef Py_ssize_t j
    for j in range(weighted_sum.shape[0]):
        if nans_array[j]+1 < 0.5:
            weighted_sum[j] = np.nan
        elif isnan(Z[j, len(Z)-1]):
            weighted_sum[j] = np.nan

    return weighted_sum


@cython.boundscheck(False)
@cython.wraparound(False)
cdef np.ndarray[np.float64_t, ndim=1] cython_dot_with_nans(np.ndarray[np.float64_t, ndim=1] Q, np.ndarray[np.float64_t, ndim=1] x):
    cdef np.ndarray[np.float64_t, ndim=1] Z = Q.copy()
    cdef np.ndarray[np.float64_t, ndim=1] nans_array = np.nansum(Z*Q+x, axis=1)
    cdef Py_ssize_t i
    for i in range(Z.shape[0]):
        Z[i, len(Z)-1] = (Z[i, len(Z)-1].T * (1/(nans_array[i]+1))).T

    cdef np.ndarray[np.float64_t, ndim=1] weighted_sum = np.nansum(Z*x, axis=1)
    cdef Py_ssize_t j
    for j in range(weighted_sum.shape[0]):
        if nans_array[j]+1 < 0.5:
            weighted_sum[j] = np.nan
        elif isnan(Z[j, len(Z)-1]):
            weighted_sum[j] = np.nan

    return weighted_sum

@cython.boundscheck(False)
@cython.wraparound(False)
cdef double cython_nanmin(np.ndarray[np.float64_t, ndim=1] x, np.ndarray[np.float64_t, ndim=1] Q, double const):
    cdef np.ndarray[np.float64_t, ndim=1] x2 = np.append(x, -1)
    cdef np.ndarray[np.float64_t, ndim=1] weighted_sum = cython_dot_with_nans(Q, x2)

    return np.nanmin(weighted_sum) - const

@cython.boundscheck(False)
@cython.wraparound(False)
cdef double cython_nanmedian(np.ndarray[np.float64_t, ndim=1] x, np.ndarray[np.float64_t, ndim=1] Q, double const):
    cdef np.ndarray[np.float64_t, ndim=1] x2 = np.append(x, -1)
    cdef np.ndarray[np.float64_t, ndim=1] weighted_sum = cython_dot_with_nans(Q, x2)

    return np.nanmedian(weighted_sum) - const

@cython.boundscheck(False)
@cython.wraparound(False)
cdef double cython_percentile_diff(np.ndarray[np.float64_t, ndim=1] x, np.ndarray[np.float64_t, ndim=1] V, np.ndarray[np.float64_t, ndim=1] s, double max_pctl):
    cdef np.ndarray[np.float64_t, ndim=1] wi = np.append(x, -1)
    cdef double ratio = np.sum(s*wi) / np.sum(V[len(V)-1]*wi)
    cdef np.ndarray[np.float64_t, ndim=1] hist_offer = cython_dot_with_nans(V, wi)*ratio
    cdef int lessthan = 0
    cdef Py_ssize_t i
    for i in range(hist_offer.shape[0]):
        if hist_offer[i] < hist_offer[len(hist_offer)-1]:
            lessthan += 1

    cdef double percentile = lessthan / len(hist_offer)

    return max_pctl - percentile


@cython.boundscheck(False)
@cython.wraparound(False)
cdef object cython_minimize(np.ndarray[np.float64_t, ndim=1] A, np.ndarray[np.float64_t, ndim=1] b, np.ndarray[np.float64_t, ndim=1] x0, list bnds, list cons):
    cdef object res = minimize(objective_function, args=(A, b), x0=x0, bounds=bnds, constraints=cons, options={'maxiter': 100})
    return res

@cython.boundscheck(False)
@cython.wraparound(False)
cdef double objective_function(np.ndarray[np.float64_t, ndim=1] x, np.ndarray[np.float64_t, ndim=1] A, np.ndarray[np.float64_t, ndim=1] b):
    cdef np.ndarray[np.float64_t, ndim=1] A_dot_x = np.dot(A, x)
    cdef double result = -np.linalg.norm(A_dot_x - b) ** 2
    return result

@cython.boundscheck(False)
@cython.wraparound(False)
cdef cython_loop(np.ndarray[np.float64_t, ndim=1] A, np.ndarray[np.float64_t, ndim=1] b, list initial_points, list bnds, list cons):
    cdef object res
    for x0 in initial_points:
        res = cython_minimize(A, b, np.asarray(x0), bnds, cons)

Content of stderr:
In file included from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /root/.cache/ipython/cython/_cython_magic_74bfba22a0cb35e85567cf4a341e0f2314d54b9b.c:1255:
/usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
/root/.cache/ipython/cython/_cython_magic_74bfba22a0cb35e85567cf4a341e0f2314d54b9b.c:5186:23: error: redefinition of ‘__pyx_f_54_cython_magic_74bfba22a0cb35e85567cf4a341e0f2314d54b9b_cython_dot_with_nans’
 5186 | static PyArrayObject *__pyx_f_54

In [2]:
%load_ext Cython